In [2]:
%matplotlib widget
import pandas as pd
import upsetplot
import matplotlib.pyplot as plt
from itertools import product

df = pd.read_csv('run_218.df',index_col=list(range(30)))

dfc = df[~df.id.str.contains('REFDEL')]
dfc = dfc[~dfc.id.str.contains('MISDEL')]

In [74]:
sorted_names= sorted(['O_peregrine', 'O_hicanu', 'N_peregrine', 'G_flye', 'G_peregrine', 'O_raven', 'P_peregrine', 'O_flye', 'N_hicanu', 'N_shasta', 'G_raven', 'O_shasta', 'N_flye', 'N_raven', 'B_flye', 'G_hifiasm', 'P_hifiasm', 'B_shasta', 'P_shasta', 'B_raven', 'P_raven', 'B_hifiasm', 'B_hicanu', 'N_hifiasm', 'O_hifiasm', 'G_hicanu', 'G_shasta', 'P_flye', 'P_hicanu', 'B_peregrine'])
print(sorted_names)

['B_flye', 'B_hicanu', 'B_hifiasm', 'B_peregrine', 'B_raven', 'B_shasta', 'G_flye', 'G_hicanu', 'G_hifiasm', 'G_peregrine', 'G_raven', 'G_shasta', 'N_flye', 'N_hicanu', 'N_hifiasm', 'N_peregrine', 'N_raven', 'N_shasta', 'O_flye', 'O_hicanu', 'O_hifiasm', 'O_peregrine', 'O_raven', 'O_shasta', 'P_flye', 'P_hicanu', 'P_hifiasm', 'P_peregrine', 'P_raven', 'P_shasta']


In [75]:
sorted_names = list('_'.join(p) for p in product(('O','B','P','N','G'),('peregrine','hicanu','hifiasm','shasta','flye','raven')))
dfx= df.reset_index()
dfx= dfx[sorted_names+['id','length']]
dfx.to_csv('run_219.df',index=False)
df2 = pd.read_csv('run_219.df',index_col=list(range(30)))
dfc = df2[~df2.id.str.contains('REFDEL')]
dfc = dfc[~dfc.id.str.contains('MISDEL')]
#dfc = pd.MultiIndex.from_frame(dfx)

In [81]:
upsetplot.plot(dfc,sort_by='cardinality',sort_categories_by=None,min_subset_size=500)
#plt.savefig('testfig.png')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

{'matrix': <AxesSubplot:>,
 'shading': <AxesSubplot:>,
 'totals': <AxesSubplot:>,
 'intersections': <AxesSubplot:ylabel='Intersection size'>}

In [14]:
intersections = upsetplot.UpSet(dfc).intersections
names = dfc.index.names



[('O', 'peregrine'), ('O', 'hicanu'), ('O', 'hifiasm')]


In [177]:
from itertools import product
taur_h = list('_'.join(p) for p in product(('O','B','P','G','N'),('peregrine','hicanu','hifiasm')))
taur_s = list('_'.join(p) for p in product(('O','B','P','G','N'),('shasta','flye','raven')))



In [191]:
slicer_h = 'gc'
slicer_s = 'gc'
for i in taur_h[2::3]:
    slicer_h += f'.xs(True,level="{i}",drop_level=False)'
    slicer_s += f'.xs(False,level="{i}",drop_level=False)'
for i in taur_s[::3]:
    slicer_h += f'.xs(False,level="{i}",drop_level=False)'
    slicer_s += f'.xs(True,level="{i}",drop_level=False)'
print(slicer_h)



gc.xs(True,level="O_hifiasm",drop_level=False).xs(True,level="B_hifiasm",drop_level=False).xs(True,level="P_hifiasm",drop_level=False).xs(True,level="G_hifiasm",drop_level=False).xs(True,level="N_hifiasm",drop_level=False).xs(False,level="O_shasta",drop_level=False).xs(False,level="B_shasta",drop_level=False).xs(False,level="P_shasta",drop_level=False).xs(False,level="G_shasta",drop_level=False).xs(False,level="N_shasta",drop_level=False)


In [192]:
g=pd.read_csv('run_10.df',index_col=list(range(10)))
gc = g[~g.id.str.contains('REFDEL')]
gc = gc[~gc.id.str.contains('MISDEL')]
gc = gc[~gc.id.str.contains('INV')]

In [195]:
h_only = eval(slicer_h)
s_only = eval(slicer_s)
print(h_only[h_only['length']>1000])
print(s_only[s_only['length']>1000])

                                                                                                      id  \
P_shasta B_shasta N_hifiasm O_shasta P_hifiasm O_hifiasm G_hifiasm B_hifiasm N_shasta G_shasta             
False    False    True      False    True      True      True      True      False    False     s11269_5   
                                                                                      False     s11272_5   
                                                                                      False     s11274_5   
                                                                                      False     s11278_5   
                                                                                      False     s11280_5   
                                                                                      False     s11283_5   
                                                                                      False     s11285_5   
                            

In [ ]:
repeat_rows = []

for i in bubbles:
    split = i.split('_')
    chrom, node = split[-1], split[0]
    fname = get_dir('SV','{chr}.L{L}.gfa',chr=chrom,**wildcards)
    search_string = f'$1=="S" && $2=="{node}" {{print ">{node}\n"$3}}'
    fout = get_dir('repeat',f'{chrom}.{node}.fa',**wildcards)
    
    shell(f'awk {search_string} {fname} > {fout}')
    shell(f'RepeatMasker -xsmall -lib {config['repeat_library']} -qq -no_is {fout}')
    row = extract_repeats(fout+'.tbl')
    row.update({'chr':chrom,'node':node})
    repeat_rows.append(row)

def extract_repeats(fname):
    elements = ('SINEs','LINEs','LTR elements', 'Satellites', 'Unclassified')
    repeats = dict()

    with open(fname,'r') as file_:
        for line in file_:
            if 'total length' in line:
                repeats['length'] = int(line.split('(')[1].split()[0])
            elif 'GC level' in line:
                repeats['GC'] = float(line.split()[2])
            elif 'bases masked' in line:
                repeats['masked'] = float(line.split()[5])
            for element in elements:
                if element in line:
                    repeats[element] = float(line.split('bp')[1].split()[0])
    return repeats

In [3]:
import seaborn as sns

In [17]:
ds = []
for i in range(300,310):
    df = pd.read_csv(f'bubbles_repeat.{i}.df')
    df['call'] = i
    ds.append(df)
df=pd.concat(ds)

In [18]:
print(df.columns)
print(df.dtypes)

Index(['length', 'GC', 'masked', 'SINEs', 'LINEs', 'LTR elements',
       'Unclassified', 'Satellites', 'data', 'chr', 'node', 'call'],
      dtype='object')
length          float64
GC              float64
masked          float64
SINEs           float64
LINEs           float64
LTR elements    float64
Unclassified    float64
Satellites      float64
data             object
chr               int64
node             object
call              int64
dtype: object


In [19]:
sns.pairplot(data=df,hue='data',corner=True,diag_kind='hist',diag_kws={'common_norm':False,'stat':'density'},grid_kws={'diag_sharey':False})


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [59]:
df[df['Satellites']>0].groupby(['data']).median()

,length,GC,masked,SINEs,LINEs,LTR elements,Unclassified,Satellites,chr,call
data,,,,,,,,,,
HiFi,2751.0,46.60,99.96,0.0,0.0,0.0,0.0,99.61,4,304
ONT,1765.0,47.25,99.89,0.0,0.0,0.0,0.0,99.89,4,303


In [21]:
import scipy.stats as ss

In [32]:
df=df.dropna()
for i in ('SINEs','LINEs','LTR elements', 'Satellites', 'Unclassified','GC','masked','length'):
    print(i)
    print(ss.mannwhitneyu(df[df['data']=='HiFi'][i],df[df['data']=='ONT'][i],alternative='less',)[1])
    print(ss.mannwhitneyu(df[df['data']=='HiFi'][i],df[df['data']=='ONT'][i],alternative='greater')[1])

SINEs
1.3851261512022712e-13
0.9999999999998618
LINEs
0.9999999996848143
3.157018513679383e-10
LTR elements
0.9994792999406322
0.0005211886120029712
Satellites
0.595245454720567
0.40501623270941634
Unclassified
0.024100417360328613
0.9759140824216059
GC
0.5366741283807454
0.4634271765312059
masked
0.9999770975122745
2.2927603531613923e-05
length
0.9999999930375888
6.972789180845565e-09


In [55]:
df[(df['data']=='ONT')&(df['call']==301)]
print(df[(df['data']=='ONT')&(df['call']==305)])

       length     GC  masked  SINEs  LINEs  LTR elements  Unclassified  \
78     3775.0  46.57   99.89   0.00   0.00          0.00          0.00   
79     1720.0  45.29   66.57   0.00   0.00          0.00         66.57   
80    21458.0  39.81   54.02   5.35  17.73          2.64         27.50   
81    76225.0  41.62   54.89   6.46  20.76          4.82         22.28   
82    31157.0  43.29   53.62   3.69  32.36          2.20         14.86   
83     5889.0  37.68   74.73   3.45   0.00          1.19         68.60   
84     1962.0  46.94   33.79   8.46   6.01         14.83          4.49   
85     1509.0  42.81   90.46  13.32   6.89          1.46         68.79   
86     2482.0  46.82   34.29  18.09   0.00          0.00         16.20   
87     3873.0  52.67   23.26   0.00   0.00         13.79          4.52   
88     1850.0  50.27   11.51  11.51   0.00          0.00          0.00   
89     3959.0  48.47    8.21   5.23   2.98          0.00          0.00   
90     5085.0  53.49   20.71   9.44   